# Create EDD Study Files From Data

This notebook creates the files needed for importing a study into Experiment Data Depot (EDD).

## Inputs and outputs

### Required file to run this notebook:
   - `../data/DBTL7/media_descriptions.csv` - media designs for each of the wells
   
   - `../data/DBTL7/OD.xlsx` - production data from the plate reader


### File generated by running this notebook:
   - `edd_experiment_description.csv`
   
   - `edd_protocol.csv`
 
    
The files are stored in the user defined directory.

## Setup

Importing needed libraries:

In [3]:
import sys
sys.path.append('../media_compiler')

import pandas as pd
import openpyxl

from core import create_media_description

### User parameters

In [4]:
CYCLE = 6.1

user_params = {
    'media_file': f'../flaviolin data/DBTL{CYCLE}/media_descriptions.csv',  
    'measurement_file': f'../flaviolin data/DBTL{CYCLE}/OD.xlsx',
    'output_file_path': f'../flaviolin data/DBTL{CYCLE}', # Folder for output files,
    'num_replicates': 3,
    'num_designs': 16,
    'protocol_name': ['OD600', 'OD340'],
    'time_point': 48,
    'part_id': 'JBx_193086',
    'media': 'MOPS',
    'culture_volume': 15,
    'well_volume': 1500,
    'shaking_speed': 800,
    'temperature': 30,
    } 


In [5]:
df = pd.read_csv(user_params['media_file'], index_col=0)
df.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l]
Well,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.018833,20.0,0.557771,0.479914,0.073016,15.011862,0.039553,662.711937,0.000262,0.002831,0.001193,0.000733,0.00004,0.05
B1,40.0,4.0,0.018833,20.0,0.557771,0.479914,0.073016,15.011862,0.039553,662.711937,0.000262,0.002831,0.001193,0.000733,0.00004,0.05
C1,40.0,4.0,0.018833,20.0,0.557771,0.479914,0.073016,15.011862,0.039553,662.711937,0.000262,0.002831,0.001193,0.000733,0.00004,0.05
D1,40.0,4.0,0.018298,20.0,0.557728,0.479902,0.071367,15.011862,0.039414,662.711936,0.000265,0.002673,0.001118,0.001638,0.00001,0.05
E1,40.0,4.0,0.018298,20.0,0.557728,0.479902,0.071367,15.011862,0.039414,662.711936,0.000265,0.002673,0.001118,0.001638,0.00001,0.05


## Create Line Description

In [6]:
df['Line Description'] = df.apply(create_media_description, axis=1)


In [7]:
df['Line Description']

Well
A1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
D3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
E3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
F3    MOPS[mM]: 40.000000, Tricine[mM]: 4.0

## Create Line Names

Add metadata for media and replicates to craft Line Names as **C[.]\_W[.]1\_[.]3-R[.]** denoting cycle number, wells occupying the same design and replicate number:

Check if it's row or column order of lines:

In [8]:
column_order = True if df.index[1][0] == 'B' else False

In [9]:
reps = user_params['num_replicates']
num_media_designs = user_params['num_designs']

def linefunction_colum_order(row):
    well = row.name
    if well[0] in 'ABC':
        return f'C{CYCLE}_WA{well[1]}_C{well[1]}-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_WD{well[1]}_F{well[1]}-R{row["Replicate"]}'                                                                                              

def linefunction_row_order(row):
    well = row.name
    if int(well[1]) < 5:
        return f'C{CYCLE}_W{well[0]}1_{well[0]}4-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_W{well[0]}5_{well[0]}8-R{row["Replicate"]}'      
    
def linefunction_row_order_triplicates(row):
    well = row.name

    well_names = list(df.index)
    index = well_names.index(well)
    rem = index%3
    name = f'C{CYCLE}_W{well_names[index-rem]}_{well_names[index-rem+2]}-R{row["Replicate"]}'
    print(name)
    return name
    
    

    
df['Replicate'] = [i+1 for _ in range(num_media_designs) for i in range(reps)]
df['Line Name'] = df.apply(linefunction_row_order_triplicates, axis=1)

# if column_order:
#     df['Line Name'] = df.apply(linefunction_colum_order, axis=1)
# else:
#     df['Line Name'] = df.apply(linefunction_row_order, axis=1)

C6.1_WA1_C1-R1
C6.1_WA1_C1-R2
C6.1_WA1_C1-R3
C6.1_WD1_F1-R1
C6.1_WD1_F1-R2
C6.1_WD1_F1-R3
C6.1_WA2_C2-R1
C6.1_WA2_C2-R2
C6.1_WA2_C2-R3
C6.1_WD2_F2-R1
C6.1_WD2_F2-R2
C6.1_WD2_F2-R3
C6.1_WA3_C3-R1
C6.1_WA3_C3-R2
C6.1_WA3_C3-R3
C6.1_WD3_F3-R1
C6.1_WD3_F3-R2
C6.1_WD3_F3-R3
C6.1_WA4_C4-R1
C6.1_WA4_C4-R2
C6.1_WA4_C4-R3
C6.1_WD4_F4-R1
C6.1_WD4_F4-R2
C6.1_WD4_F4-R3
C6.1_WA5_C5-R1
C6.1_WA5_C5-R2
C6.1_WA5_C5-R3
C6.1_WD5_F5-R1
C6.1_WD5_F5-R2
C6.1_WD5_F5-R3
C6.1_WA6_C6-R1
C6.1_WA6_C6-R2
C6.1_WA6_C6-R3
C6.1_WD6_F6-R1
C6.1_WD6_F6-R2
C6.1_WD6_F6-R3
C6.1_WA7_C7-R1
C6.1_WA7_C7-R2
C6.1_WA7_C7-R3
C6.1_WD7_F7-R1
C6.1_WD7_F7-R2
C6.1_WD7_F7-R3
C6.1_WA8_C8-R1
C6.1_WA8_C8-R2
C6.1_WA8_C8-R3
C6.1_WD8_F8-R1
C6.1_WD8_F8-R2
C6.1_WD8_F8-R3


In [11]:
df

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name
Well,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.018833,20.0,0.557771,0.479914,0.073016,15.011862,0.039553,662.711937,0.000262,0.002831,0.001193,0.000733,0.000040,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C6.1_WA1_C1-R1
B1,40.0,4.0,0.018833,20.0,0.557771,0.479914,0.073016,15.011862,0.039553,662.711937,0.000262,0.002831,0.001193,0.000733,0.000040,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C6.1_WA1_C1-R2
C1,40.0,4.0,0.018833,20.0,0.557771,0.479914,0.073016,15.011862,0.039553,662.711937,0.000262,0.002831,0.001193,0.000733,0.000040,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C6.1_WA1_C1-R3
D1,40.0,4.0,0.018298,20.0,0.557728,0.479902,0.071367,15.011862,0.039414,662.711936,0.000265,0.002673,0.001118,0.001638,0.000010,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C6.1_WD1_F1-R1
E1,40.0,4.0,0.018298,20.0,0.557728,0.479902,0.071367,15.011862,0.039414,662.711936,0.000265,0.002673,0.001118,0.001638,0.000010,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C6.1_WD1_F1-R2
F1,40.0,4.0,0.018298,20.0,0.557728,0.479902,0.071367,15.011862,0.039414,662.711936,0.000265,0.002673,0.001118,0.001638,0.000010,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C6.1_WD1_F1-R3
A2,40.0,4.0,0.021191,20.0,0.747817,0.700241,0.063038,16.489651,0.105661,470.249173,0.000246,0.002149,0.001422,0.002613,0.000227,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C6.1_WA2_C2-R1
B2,40.0,4.0,0.021191,20.0,0.747817,0.700241,0.063038,16.489651,0.105661,470.249173,0.000246,0.002149,0.001422,0.002613,0.000227,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C6.1_WA2_C2-R2
C2,40.0,4.0,0.021191,20.0,0.747817,0.700241,0.063038,16.489651,0.105661,470.249173,0.000246,0.002149,0.001422,0.002613,0.000227,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C6.1_WA2_C2-R3


## Process measurement files

Read measurements file:

In [12]:
df_600 = pd.read_excel(user_params['measurement_file'], sheet_name='600', index_col=0)
df_340 = pd.read_excel(user_params['measurement_file'], sheet_name='340', index_col=0)

df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.0699,0.9280,0.7896,1.0238,0.9137,0.9587,0.9270,0.9534,0.0358,0.0449,0.0447,0.0440
B,0.0658,0.8751,0.8682,1.0887,1.0195,0.8880,0.9708,0.9253,0.0419,0.0411,0.0416,0.0400
C,0.0633,0.9070,0.9080,1.0489,0.9231,0.9576,1.0028,0.9277,0.0353,0.0363,0.0350,0.0353
D,0.0549,1.0102,0.8667,0.8469,0.9118,0.9652,0.9830,0.1671,0.0297,0.0299,0.0296,0.0288
E,0.0549,1.0095,0.8243,0.8774,0.9125,0.9455,0.9903,0.2725,0.0250,0.0250,0.0247,0.0239
F,0.0535,0.9488,0.8263,0.8161,0.8586,0.9062,0.9521,0.0542,0.0214,0.0220,0.0209,0.0213
G,0.0207,0.0220,0.0214,0.0215,0.0215,0.0214,0.0211,0.0214,0.0210,0.0209,0.0204,0.0203
H,0.0219,0.0219,0.0217,0.0219,0.0221,0.0216,0.0213,0.0213,0.0217,0.0215,0.0213,0.0211


In [13]:
df_340

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.1751,0.5667,0.5535,0.6299,0.6309,0.5983,0.5450,0.6311,0.1060,0.1012,0.1004,0.0978
B,0.1717,0.5240,0.5211,0.6066,0.5663,0.5460,0.5619,0.6180,0.1014,0.0992,0.0977,0.0931
C,0.1663,0.5260,0.5063,0.5980,0.5465,0.5825,0.5556,0.6078,0.0930,0.0932,0.0906,0.0886
D,0.1576,0.5439,0.5195,0.5614,0.5234,0.5641,0.5685,0.3506,0.0870,0.0856,0.0843,0.0801
E,0.1548,0.5777,0.5304,0.5781,0.5428,0.6229,0.6083,0.3602,0.0806,0.0802,0.0787,0.0749
F,0.1508,0.6133,0.5555,0.5933,0.5949,0.6501,0.6584,0.3509,0.0771,0.0760,0.0746,0.0720
G,0.0710,0.0744,0.0770,0.0776,0.0793,0.0790,0.0786,0.0786,0.0770,0.0754,0.0723,0.0702
H,0.0713,0.0728,0.0749,0.0761,0.0746,0.0740,0.0748,0.0745,0.0749,0.0737,0.0723,0.0705


### Process OD600

Normalize the data to the control well A9 value (water content):

In [14]:
zero_value_600 = df_600.at['A', 9]
df_600.loc[:, df_600.columns] -= zero_value_600
df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.0341,0.8922,0.7538,0.9880,0.8779,0.9229,0.8912,0.9176,0.0000,0.0091,0.0089,0.0082
B,0.0300,0.8393,0.8324,1.0529,0.9837,0.8522,0.9350,0.8895,0.0061,0.0053,0.0058,0.0042
C,0.0275,0.8712,0.8722,1.0131,0.8873,0.9218,0.9670,0.8919,-0.0005,0.0005,-0.0008,-0.0005
D,0.0191,0.9744,0.8309,0.8111,0.8760,0.9294,0.9472,0.1313,-0.0061,-0.0059,-0.0062,-0.0070
E,0.0191,0.9737,0.7885,0.8416,0.8767,0.9097,0.9545,0.2367,-0.0108,-0.0108,-0.0111,-0.0119
F,0.0177,0.9130,0.7905,0.7803,0.8228,0.8704,0.9163,0.0184,-0.0144,-0.0138,-0.0149,-0.0145
G,-0.0151,-0.0138,-0.0144,-0.0143,-0.0143,-0.0144,-0.0147,-0.0144,-0.0148,-0.0149,-0.0154,-0.0155
H,-0.0139,-0.0139,-0.0141,-0.0139,-0.0137,-0.0142,-0.0145,-0.0145,-0.0141,-0.0143,-0.0145,-0.0147


Set negative values to zero, multiply the values by 10 to account for 10x dilution and keep only 8 columns and 6 rows:

In [15]:
df_600[df_600 < 0] = 0
df_600.loc[:, df_600.columns] *= 10
df_600 = df_600.iloc[:6,:8]
df_600

,1,2,3,4,5,6,7,8
A,0.341,8.922,7.538,9.880,8.779,9.229,8.912,9.176
B,0.300,8.393,8.324,10.529,9.837,8.522,9.350,8.895
C,0.275,8.712,8.722,10.131,8.873,9.218,9.670,8.919
D,0.191,9.744,8.309,8.111,8.760,9.294,9.472,1.313
E,0.191,9.737,7.885,8.416,8.767,9.097,9.545,2.367
F,0.177,9.130,7.905,7.803,8.228,8.704,9.163,0.184


### Process OD340

Do the same for OD340, except of multiplication by 10, as the OD340 measurements were taken from non-diluted samples:

In [16]:
zero_value_340 = df_340.at['A', 9]
df_340.loc[:, df_340.columns] -= zero_value_340
df_340[df_340 < 0] = 0
df_340 = df_340.iloc[:6,:8]
df_340

,1,2,3,4,5,6,7,8
A,0.0691,0.4607,0.4475,0.5239,0.5249,0.4923,0.4390,0.5251
B,0.0657,0.4180,0.4151,0.5006,0.4603,0.4400,0.4559,0.5120
C,0.0603,0.4200,0.4003,0.4920,0.4405,0.4765,0.4496,0.5018
D,0.0516,0.4379,0.4135,0.4554,0.4174,0.4581,0.4625,0.2446
E,0.0488,0.4717,0.4244,0.4721,0.4368,0.5169,0.5023,0.2542
F,0.0448,0.5073,0.4495,0.4873,0.4889,0.5441,0.5524,0.2449


Transform wide to long format, matching the order of wells in index to the one from `df`:

In [17]:
if column_order:
    indfcn = lambda x: f'{x.name}{int(x["variable"])}'
    # df_600 = df_600.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.melt(ignore_index=False, value_name='OD340')
else:
    indfcn = lambda x: f'{x["variable"]}{(x.name)}'
    # df_600 = df_600.T.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.T.melt(ignore_index=False, value_name='OD340')

# df_600['Well'] = df_600.apply(indfcn, axis=1)
df_340['Well'] = df_340.apply(indfcn, axis=1)

# df_600.index = df_600['Well']
df_340.index = df_340['Well']

# df_600.drop(columns=['variable', 'Well'], inplace=True)
df_340.drop(columns=['variable', 'Well'], inplace=True)

# df_600.head()

Add measurements to the main dataframe:

In [18]:
# measurOD600 = user_params['protocol_name'][0]
measurOD340 = user_params['protocol_name'][1]
# df[measurOD600] = df_600
df[measurOD340] = df_340
df.head(2)


,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name,OD340
Well,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.018833,20.0,0.557771,0.479914,0.073016,15.011862,0.039553,662.711937,0.000262,0.002831,0.001193,0.000733,0.00004,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C6.1_WA1_C1-R1,0.0691
B1,40.0,4.0,0.018833,20.0,0.557771,0.479914,0.073016,15.011862,0.039553,662.711937,0.000262,0.002831,0.001193,0.000733,0.00004,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C6.1_WA1_C1-R2,0.0657


## Create EDD Experiment Description File

In [19]:
df['Media'] = user_params['media']
df['Part ID'] = user_params['part_id']
df['Culture Volume'] = user_params['culture_volume']
df['Flask Volume'] = user_params['well_volume']
df['Growth Temperature'] = user_params['temperature']
df['Shaking speed'] = user_params['shaking_speed']
# df['Starting OD'] =
# df['Replicate Count'] = 24


In [20]:
# Invalid columns for now in EDD
# df['Humidity[%]'] = user_params['humidity']
# df['Plate'] = user_params['plate']

In [21]:
exp_descr_file = f'{user_params["output_file_path"]}/edd_experiment_description.xlsx'
df[['Line Name',
    'Line Description',
    'Part ID',
    'Media',
    'Culture Volume',
    'Flask Volume',
    'Growth Temperature',
    'Shaking speed',
]].to_excel(exp_descr_file, index=False)

## Create EDD Measurement File

OD600

In [22]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD600}.xlsx'
# df['Measurement Type'] = 'Optical Density'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD600]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD340

In [23]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD340}.xlsx'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD340]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD

In [24]:
measurement_file = f'{user_params["output_file_path"]}/edd_OD.xlsx'
df['Measurement Type'] = measurOD340

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD340]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

In [25]:
df['Kan[g/l]']

Well
A1    0.05
B1    0.05
C1    0.05
D1    0.05
E1    0.05
F1    0.05
A2    0.05
B2    0.05
C2    0.05
D2    0.05
E2    0.05
F2    0.05
A3    0.05
B3    0.05
C3    0.05
D3    0.05
E3    0.05
F3    0.05
A4    0.05
B4    0.05
C4    0.05
D4    0.05
E4    0.05
F4    0.05
A5    0.05
B5    0.05
C5    0.05
D5    0.05
E5    0.05
F5    0.05
A6    0.05
B6    0.05
C6    0.05
D6    0.05
E6    0.05
F6    0.05
A7    0.05
B7    0.05
C7    0.05
D7    0.05
E7    0.05
F7    0.05
A8    0.05
B8    0.05
C8    0.05
D8    0.05
E8    0.05
F8    0.05
Name: Kan[g/l], dtype: float64